#1. 구글드라이브와 연동

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#2. StyleGan2-ADA 초기 환경 세팅

- StyleGan2-ADA는 텐서플로우 Tensorflow 1에서만 작동한다.
- GPU환경 필요 : - colab에서 실행시 런타임 유형에서 gpu환경설정

In [ ]:
##Tensorflow 1로 버전 설정

%tensorflow_version 1.x
import tensorflow as tf
tf.__version__

TensorFlow 1.x selected.


'1.15.2'

In [ ]:
##GPU환경 확인

!nvidia-smi

Thu Jun 10 03:18:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------


#3. StyleGan2-ADA 설치
- !git clone https://github.com/dvschultz/stylegan2-ada


In [ ]:
## StyleGan2-ADA repository의 내용을 특정 폴더에 저장

import os
if os.path.isdir("/content/drive/My Drive/colab-sg2-ada"):
    %cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
else:
    #install script
    %cd "/content/drive/My Drive/"
    !mkdir colab-sg2-ada
    %cd colab-sg2-ada
    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

/content/drive/My Drive/colab-sg2-ada/stylegan2-ada


#4. 데이터셋 사이즈 변형 

- StyleGan2-ADA는 정사각형 형태의 일관된 크기의 이미지만 학습한다.(128*128, 256*256, 1024*1024)
- 정사각형 형태의 이미지로 바꾸는 방법은 https://www.youtube.com/watch?v=HgSfKfBMAaY&t=338s 참고하면 된다.
- 이번 학습에서는 128*128크기의 이미지로 변형했다.


#5. 사이즈 변형된 데이터셋을 `.tfrecords` 형식으로 변형

In [ ]:
##데이터셋의 압축 해제

zip_path = "/content/drive/MyDrive/코드스테이츠/개인프로젝트/section6/COVID_19_classified_forGANv4.zip"

!unzip {zip_path} -d /content/

Archive:  /content/drive/MyDrive/코드스테이츠/개인프로젝트/section6/COVID_19_classified_forGANv4.zip
  inflating: /content/COVID_19_classified_forGANv4/COVID-100.png  
  inflating: /content/COVID_19_classified_forGANv4/COVID-1000.png  
  inflating: /content/COVID_19_classified_forGANv4/COVID-1001.png  
  inflating: /content/COVID_19_classified_forGANv4/COVID-1002.png  
  inflating: /content/COVID_19_classified_forGANv4/COVID-1003.png  
  inflating: /content/COVID_19_classified_forGANv4/COVID-1004.png  
  inflating: /content/COVID_19_classified_forGANv4/COVID-1005.png  
  inflating: /content/COVID_19_classified_forGANv4/COVID-1006.png  
  inflating: /content/COVID_19_classified_forGANv4/COVID-1007.png  
  inflating: /content/COVID_19_classified_forGANv4/COVID-1008.png  
  inflating: /content/COVID_19_classified_forGANv4/COVID-1009.png  
  inflating: /content/COVID_19_classified_forGANv4/COVID-101.png  
  inflating: /content/COVID_19_classified_forGANv4/COVID-1010.png  
  inflating: /c

In [ ]:
##데이터셋을 .tfrecords 형식으로 변형

#update this to the path to your image folder
dataset_path = "/content/COVID_19_classified_forGANv4"
#give your dataset a name
dataset_name = "COVID_19"

#you don't need to edit anything here
!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}

#6. Training StyleGan2-ADA
- 이전 모델에 이어서 학습을 원할경우 resume_from에 pkl파일 경로를 넣어준다.

In [ ]:
##학습 중간에 만들어지는 모델과 스냅샷은 '/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results'에 저장됨

#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "COVID_19"
#how often should the model generate samples and a .pkl file
snapshot_count = 2
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = "bg"

#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)
# resuming? get the path to your latest .pkl file and use that
resume_from = "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00004-COVID_19-mirror-paper256-bg-resumecustom/network-snapshot-002384.pkl"
#don't edit this unless you know what you're doing :)
!python train.py --outdir ./results --snap={snapshot_count} --cfg=paper256 --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --resume={resume_from} --augpipe="bg"

tcmalloc: large alloc 4294967296 bytes == 0x55d4a42ae000 @  0x7f3321205001 0x7f331e40854f 0x7f331e458b58 0x7f331e45cb17 0x7f331e4fb203 0x55d49ce33d54 0x55d49ce33a50 0x55d49cea8105 0x55d49cea27ad 0x55d49ce35c9f 0x55d49ce76d79 0x55d49ce73cc4 0x55d49ce34559 0x55d49cea84f8 0x55d49cea24ae 0x55d49cd74e2c 0x55d49cea4bb5 0x55d49cea24ae 0x55d49cd74e2c 0x55d49cea4bb5 0x55d49cea27ad 0x55d49cd74e2c 0x55d49cea4bb5 0x55d49ce3530a 0x55d49cea33b5 0x55d49cea24ae 0x55d49cea21b3 0x55d49cf6c182 0x55d49cf6c4fd 0x55d49cf6c3a6 0x55d49cf43723
tcmalloc: large alloc 4294967296 bytes == 0x55d5a42ae000 @  0x7f33212031e7 0x7f331e40846e 0x7f331e458c7b 0x7f331e45935f 0x7f331e4fb103 0x55d49ce33d54 0x55d49ce33a50 0x55d49cea8105 0x55d49cea24ae 0x55d49ce353ea 0x55d49cea432a 0x55d49cea24ae 0x55d49ce353ea 0x55d49cea432a 0x55d49cea24ae 0x55d49ce353ea 0x55d49cea432a 0x55d49ce3530a 0x55d49cea33b5 0x55d49cea24ae 0x55d49ce353ea 0x55d49cea77f0 0x55d49cea24ae 0x55d49ce353ea 0x55d49cea432a 0x55d49cea27ad 0x55d49ce35c9f 0x55d49ce7